In [1]:
import requests
#import warnings
#warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Making HTTPS Request

In [2]:
# https://aphorisms.glitch.me returns a random quotation
response = requests.get('http://aphorisms.glitch.me/')
response


<Response [200]>

In [3]:
response.ok


True

In [4]:
response.status_code

200

In [5]:
response.text

'{"quote":"Learning a skill requires practicing that skill","author":"Luis Montealegre"}'

# Interacting with JSON

* Here we see that the repsonse we got back contains a JSON object (we could also verify this by visiting the URL in a web browser).

In [6]:
url = 'https://swapi.dev/api/people/5'
response = requests.get(url)
print(response.text)

{"name":"Leia Organa","height":"150","mass":"49","hair_color":"brown","skin_color":"light","eye_color":"brown","birth_year":"19BBY","gender":"female","homeworld":"https://swapi.dev/api/planets/2/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"species":[],"vehicles":["https://swapi.dev/api/vehicles/30/"],"starships":[],"created":"2014-12-10T15:20:09.791000Z","edited":"2014-12-20T21:17:50.315000Z","url":"https://swapi.dev/api/people/5/"}


* Since the response is JSON, we can use the .json method on the response object to get a data structure we can work with:

In [7]:
data = response.json()
print(type(data))
data

<class 'dict'>


{'name': 'Leia Organa',
 'height': '150',
 'mass': '49',
 'hair_color': 'brown',
 'skin_color': 'light',
 'eye_color': 'brown',
 'birth_year': '19BBY',
 'gender': 'female',
 'homeworld': 'https://swapi.dev/api/planets/2/',
 'films': ['https://swapi.dev/api/films/1/',
  'https://swapi.dev/api/films/2/',
  'https://swapi.dev/api/films/3/',
  'https://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['https://swapi.dev/api/vehicles/30/'],
 'starships': [],
 'created': '2014-12-10T15:20:09.791000Z',
 'edited': '2014-12-20T21:17:50.315000Z',
 'url': 'https://swapi.dev/api/people/5/'}

* Now we have a dictionary that we can work with.

* This API provides the availabe resources when we call the root url, so let's make a request so that we can take a look at it.

In [8]:
root_url = 'https://swapi.dev/api/'
response = requests.get(root_url)
print(response)
response.json()

<Response [200]>


{'people': 'https://swapi.dev/api/people/',
 'planets': 'https://swapi.dev/api/planets/',
 'films': 'https://swapi.dev/api/films/',
 'species': 'https://swapi.dev/api/species/',
 'vehicles': 'https://swapi.dev/api/vehicles/',
 'starships': 'https://swapi.dev/api/starships/'}

* Based on this, let's take a look at the people. We'll make our request, and explore the shape of the response that we get back.

In [9]:
response = requests.get('https://swapi.dev/api/people/')
data = response.json()
data.keys()


dict_keys(['count', 'next', 'previous', 'results'])

# People 

In [10]:
number_of_people = data['count']
next_page = data['next']
previous_page = data['previous']

print(f'number_of_people: {number_of_people}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')

number_of_people: 82
next_page: https://swapi.dev/api/people/?page=2
previous_page: None


* Here the response has some built-in properties that tell us how to get the next page and the previous one. We can see that this API uses an addon to the end of the url that allows us to pick which page of results we want to look at.

* Once we've drilled down into the data structure, we'll find that the entire response is a sort of wrapper around the results property:

In [11]:
data['results'][:2]

[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['https://swapi.dev/api/vehicles/14/',
   'https://swapi.dev/api/vehicles/30/'],
  'starships': ['https://swapi.dev/api/starships/12/',
   'https://swapi.dev/api/starships/22/'],
  'created': '2014-12-09T13:50:51.644000Z',
  'edited': '2014-12-20T21:17:56.891000Z',
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
 

* So we should be able to use the count of people and number of results to calculate what the last page number would be.

In [12]:
number_of_results = len(data['results'])
max_page = math.ceil(number_of_people / number_of_results)

print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')


number_of_results: 10
max_page: 9


* We can also turn results data into a pandas dataframe:

In [13]:
df = pd.DataFrame(data['results'])
df.head()


,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/


* Now that we've gotten the data from the first page, we can extract the data from the next page (as indicated by the API's response), and add it onto our dataframe:

In [14]:
response = requests.get(data['next'])
data = response.json()

number_of_people = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_people / number_of_results)

print(f'number_of_people: {number_of_people}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])]).reset_index()

number_of_people: 82
next_page: https://swapi.dev/api/people/?page=3
previous_page: https://swapi.dev/api/people/?page=1
number_of_results: 10
max_page: 9


* We'll repeat the process one more time:

In [15]:
response = requests.get(data['next'])
data = response.json()

number_of_people = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_people / number_of_results)

print(f'number_of_people: {number_of_people}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])]).reset_index()


number_of_people: 82
next_page: https://swapi.dev/api/people/?page=4
previous_page: https://swapi.dev/api/people/?page=2
number_of_results: 10
max_page: 9


* Let's look ahead and see what the last page looks like.

In [16]:
url = 'https://swapi.dev/api/people/?page=9'
response = requests.get(url)
data = response.json()

number_of_people = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])

print(f'number_of_people: {number_of_people}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')



number_of_people: 82
next_page: None
previous_page: https://swapi.dev/api/people/?page=8
number_of_results: 2


* Now that the API says that the next_page is None, we know we can stop making requests, and assume that we have all of the people data.


In [17]:
people = df

In [18]:
filename = "people.csv"
people.to_csv(filename)

# Exercise 

# Planets

In [19]:
url = 'https://swapi.dev/api/planets/5'
response = requests.get(url)
print(response.text)

{"name":"Dagobah","rotation_period":"23","orbital_period":"341","diameter":"8900","climate":"murky","gravity":"N/A","terrain":"swamp, jungles","surface_water":"8","population":"unknown","residents":[],"films":["https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"created":"2014-12-10T11:42:22.590000Z","edited":"2014-12-20T20:58:18.425000Z","url":"https://swapi.dev/api/planets/5/"}


In [20]:
data = response.json()
print(type(data))
data

<class 'dict'>


{'name': 'Dagobah',
 'rotation_period': '23',
 'orbital_period': '341',
 'diameter': '8900',
 'climate': 'murky',
 'gravity': 'N/A',
 'terrain': 'swamp, jungles',
 'surface_water': '8',
 'population': 'unknown',
 'residents': [],
 'films': ['https://swapi.dev/api/films/2/',
  'https://swapi.dev/api/films/3/',
  'https://swapi.dev/api/films/6/'],
 'created': '2014-12-10T11:42:22.590000Z',
 'edited': '2014-12-20T20:58:18.425000Z',
 'url': 'https://swapi.dev/api/planets/5/'}

In [21]:
root_url = 'https://swapi.dev/api/'
response = requests.get(root_url)
print(response)
response.json()

<Response [200]>


{'people': 'https://swapi.dev/api/people/',
 'planets': 'https://swapi.dev/api/planets/',
 'films': 'https://swapi.dev/api/films/',
 'species': 'https://swapi.dev/api/species/',
 'vehicles': 'https://swapi.dev/api/vehicles/',
 'starships': 'https://swapi.dev/api/starships/'}

In [22]:
response = requests.get('https://swapi.dev/api/planets/')
data = response.json()
data.keys()

dict_keys(['count', 'next', 'previous', 'results'])

In [23]:
number_of_planets = data['count']
next_page = data['next']
previous_page = data['previous']

print(f'number_of_people: {number_of_planets}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')

number_of_people: 60
next_page: https://swapi.dev/api/planets/?page=2
previous_page: None


In [24]:
data['results'][:2]

[{'name': 'Tatooine',
  'rotation_period': '23',
  'orbital_period': '304',
  'diameter': '10465',
  'climate': 'arid',
  'gravity': '1 standard',
  'terrain': 'desert',
  'surface_water': '1',
  'population': '200000',
  'residents': ['https://swapi.dev/api/people/1/',
   'https://swapi.dev/api/people/2/',
   'https://swapi.dev/api/people/4/',
   'https://swapi.dev/api/people/6/',
   'https://swapi.dev/api/people/7/',
   'https://swapi.dev/api/people/8/',
   'https://swapi.dev/api/people/9/',
   'https://swapi.dev/api/people/11/',
   'https://swapi.dev/api/people/43/',
   'https://swapi.dev/api/people/62/'],
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/4/',
   'https://swapi.dev/api/films/5/',
   'https://swapi.dev/api/films/6/'],
  'created': '2014-12-09T13:50:49.641000Z',
  'edited': '2014-12-20T20:58:18.411000Z',
  'url': 'https://swapi.dev/api/planets/1/'},
 {'name': 'Alderaan',
  'rotation_period': '24',
  'orb

In [25]:
number_of_results = len(data['results'])
max_page = math.ceil(number_of_planets / number_of_results)

print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

number_of_results: 10
max_page: 6


In [26]:
df = pd.DataFrame(data['results'])
df.head()

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/
2,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/
3,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[],[https://swapi.dev/api/films/2/],2014-12-10T11:39:13.934000Z,2014-12-20T20:58:18.423000Z,https://swapi.dev/api/planets/4/
4,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,[],"[https://swapi.dev/api/films/2/, https://swapi...",2014-12-10T11:42:22.590000Z,2014-12-20T20:58:18.425000Z,https://swapi.dev/api/planets/5/


In [27]:
response = requests.get(data['next'])
data = response.json()

number_of_planets = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_planets / number_of_results)

print(f'number_of_people: {number_of_planets}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])]).reset_index()

number_of_people: 60
next_page: https://swapi.dev/api/planets/?page=3
previous_page: https://swapi.dev/api/planets/?page=1
number_of_results: 10
max_page: 6


In [28]:
response = requests.get(data['next'])
data = response.json()

number_of_planets = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_planets / number_of_results)

print(f'number_of_planets: {number_of_planets}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])]).reset_index()

number_of_planets: 60
next_page: https://swapi.dev/api/planets/?page=4
previous_page: https://swapi.dev/api/planets/?page=2
number_of_results: 10
max_page: 6


In [29]:
response = requests.get(data['next'])
data = response.json()

number_of_planets = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_planets / number_of_results)

print(f'number_of_planets: {number_of_planets}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])])

number_of_planets: 60
next_page: https://swapi.dev/api/planets/?page=5
previous_page: https://swapi.dev/api/planets/?page=3
number_of_results: 10
max_page: 6


In [30]:
response = requests.get(data['next'])
data = response.json()

number_of_planets = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_planets / number_of_results)

print(f'number_of_planets: {number_of_planets}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])])

number_of_planets: 60
next_page: https://swapi.dev/api/planets/?page=6
previous_page: https://swapi.dev/api/planets/?page=4
number_of_results: 10
max_page: 6


In [31]:
response = requests.get(data['next'])
data = response.json()

number_of_planets = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_planets / number_of_results)

print(f'number_of_planets: {number_of_planets}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])])

number_of_planets: 60
next_page: None
previous_page: https://swapi.dev/api/planets/?page=5
number_of_results: 10
max_page: 6


In [32]:
planets = df

In [33]:
filename = "planets.csv"
planets.to_csv(filename)

# Starships

In [34]:
url = 'https://swapi.dev/api/starships/5'
response = requests.get(url)
print(response.text)

{"name":"Sentinel-class landing craft","model":"Sentinel-class landing craft","manufacturer":"Sienar Fleet Systems, Cyngus Spaceworks","cost_in_credits":"240000","length":"38","max_atmosphering_speed":"1000","crew":"5","passengers":"75","cargo_capacity":"180000","consumables":"1 month","hyperdrive_rating":"1.0","MGLT":"70","starship_class":"landing craft","pilots":[],"films":["https://swapi.dev/api/films/1/"],"created":"2014-12-10T15:48:00.586000Z","edited":"2014-12-20T21:23:49.873000Z","url":"https://swapi.dev/api/starships/5/"}


In [35]:
data = response.json()
print(type(data))
data

<class 'dict'>


{'name': 'Sentinel-class landing craft',
 'model': 'Sentinel-class landing craft',
 'manufacturer': 'Sienar Fleet Systems, Cyngus Spaceworks',
 'cost_in_credits': '240000',
 'length': '38',
 'max_atmosphering_speed': '1000',
 'crew': '5',
 'passengers': '75',
 'cargo_capacity': '180000',
 'consumables': '1 month',
 'hyperdrive_rating': '1.0',
 'MGLT': '70',
 'starship_class': 'landing craft',
 'pilots': [],
 'films': ['https://swapi.dev/api/films/1/'],
 'created': '2014-12-10T15:48:00.586000Z',
 'edited': '2014-12-20T21:23:49.873000Z',
 'url': 'https://swapi.dev/api/starships/5/'}

In [62]:
root_url = 'https://swapi.dev/api/'
response = requests.get(root_url)
print(response)
response.json()
home_content=response.json()


<Response [200]>


'https://swapi.dev/api/people/'

In [ ]:
home_content['people']
first_people_payload = requests.get(home_content['people'])


In [37]:
response = requests.get('https://swapi.dev/api/starships/')
data = response.json()
data.keys()

dict_keys(['count', 'next', 'previous', 'results'])

In [38]:
number_of_starships = data['count']
next_page = data['next']
previous_page = data['previous']

print(f'number_of_starships: {number_of_starships}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')

number_of_starships: 36
next_page: https://swapi.dev/api/starships/?page=2
previous_page: None


In [39]:
data['results'][:2]

[{'name': 'CR90 corvette',
  'model': 'CR90 corvette',
  'manufacturer': 'Corellian Engineering Corporation',
  'cost_in_credits': '3500000',
  'length': '150',
  'max_atmosphering_speed': '950',
  'crew': '30-165',
  'passengers': '600',
  'cargo_capacity': '3000000',
  'consumables': '1 year',
  'hyperdrive_rating': '2.0',
  'MGLT': '60',
  'starship_class': 'corvette',
  'pilots': [],
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'created': '2014-12-10T14:20:33.369000Z',
  'edited': '2014-12-20T21:23:49.867000Z',
  'url': 'https://swapi.dev/api/starships/2/'},
 {'name': 'Star Destroyer',
  'model': 'Imperial I-class Star Destroyer',
  'manufacturer': 'Kuat Drive Yards',
  'cost_in_credits': '150000000',
  'length': '1,600',
  'max_atmosphering_speed': '975',
  'crew': '47,060',
  'passengers': 'n/a',
  'cargo_capacity': '36000000',
  'consumables': '2 years',
  'hyperdrive_rating': '2.0',
  'MGLT': '60',
  

In [40]:
number_of_results = len(data['results'])
max_page = math.ceil(number_of_starships / number_of_results)

print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

number_of_results: 10
max_page: 4


In [41]:
df = pd.DataFrame(data['results'])
df.head()

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,30-165,600,3000000,1 year,2.0,60,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/
1,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:08:19.848000Z,2014-12-20T21:23:49.870000Z,https://swapi.dev/api/starships/3/
2,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,[],[https://swapi.dev/api/films/1/],2014-12-10T15:48:00.586000Z,2014-12-20T21:23:49.873000Z,https://swapi.dev/api/starships/5/
3,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,"342,953","843,342",1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[https://swapi.dev/api/films/1/],2014-12-10T16:36:50.509000Z,2014-12-20T21:26:24.783000Z,https://swapi.dev/api/starships/9/
4,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:59:45.094000Z,2014-12-20T21:23:49.880000Z,https://swapi.dev/api/starships/10/


In [42]:
response = requests.get(data['next'])
data = response.json()

number_of_starships = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_starships / number_of_results)

print(f'number_of_starships: {number_of_starships}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])]).reset_index()

number_of_starships: 36
next_page: https://swapi.dev/api/starships/?page=3
previous_page: https://swapi.dev/api/starships/?page=1
number_of_results: 10
max_page: 4


In [43]:
response = requests.get(data['next'])
data = response.json()

number_of_starships = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_starships / number_of_results)

print(f'number_of_starships: {number_of_starships}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])]).reset_index()

number_of_starships: 36
next_page: https://swapi.dev/api/starships/?page=4
previous_page: https://swapi.dev/api/starships/?page=2
number_of_results: 10
max_page: 4


In [44]:
response = requests.get(data['next'])
data = response.json()

number_of_starships = data['count']
next_page = data['next']
previous_page = data['previous']
number_of_results = len(data['results'])
max_page = math.ceil(number_of_starships / number_of_results)

print(f'number_of_starships: {number_of_starships}')
print(f'next_page: {next_page}')
print(f'previous_page: {previous_page}')
print(f'number_of_results: {number_of_results}')
print(f'max_page: {max_page}')

df = pd.concat([df, pd.DataFrame(data['results'])])

number_of_starships: 36
next_page: None
previous_page: https://swapi.dev/api/starships/?page=3
number_of_results: 6
max_page: 6


In [45]:
starships = df

In [46]:
filename = "starships.csv"
starships.to_csv(filename)

In [47]:
starships = starships.rename(columns={"name": "starship_name", "films" :"s_films", 
                          "created":"s_created","edited":"s_edited","url":"s_url"})

In [48]:
people = people.rename(columns={"name": "people_name", "films" :"people_films", 
                          "created":"people_created","edited":"people_edited","url":"people_url"})

In [49]:
planets = planets.rename(columns={"name": "planet_name", "films" :"planet_films", 
                          "created":"planet_created","edited":"planet_edited","url":"planet_url"})

In [50]:
planets.shape

(60, 16)

In [51]:
people.shape

(30, 18)

In [52]:
starships.shape

(36, 20)

In [53]:
horizontal_concat = pd.concat([people, starships, planets])

In [54]:
df = horizontal_concat

In [55]:
df.planet_name

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
        ...    
5         Skako
6    Muunilinst
7         Shili
8         Kalee
9        Umbara
Name: planet_name, Length: 126, dtype: object

In [56]:
filename = 'merged_starwars.csv'
df.to_csv(filename)